# Effect of expertise

- study of response entropy across participants

In [ ]:
# intro, defaults (shared across notebooks)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pingouin as pg
%matplotlib inline

# ---------------------------------------------------
# set global properties for plotting

font = {"family":"Adapter Mono PE", "size":"10", "weight":"medium"}
figure = {"titlesize":"10","titleweight":"medium"}
axes = {"titlesize":"10", "titleweight":"medium", "labelsize":"10", "labelweight":"medium"}
plt.rc("font", **font)
plt.rc("figure", **figure)
plt.rc("axes", **axes)
box_colors = dict(boxes="Black", whiskers="0.5", medians="Black", caps="0.5")

In [116]:
# Get data & fix types

data = pd.read_csv("csv/raw-data-preprocessed_tall.csv", index_col=0, header=0, dtype="unicode")
data.sort_index(axis=1, inplace=True)
data["pid"] = data["pid"].astype("int")
data["order"] = data["order"].astype("float").astype("int")
#data["response"] = data["response"].astype("category")
data["fluent in script"] = (data["fluent in script"] == "True").astype("category")
data["native in script"] = (data["native in script"] == "True").astype("category")

In [117]:
display(data)
pcount = len(data[data["order"] == 1]) // 56
scount = len(data) // 56
print(f"Imported {data.shape[0]} rows, {data.shape[1]} columns, {pcount} participants, {scount} sessions, 56 trial responses each.")

,age,date,design skills,fluent in script,fluent languages,native in script,native languages,order,pid,reading skills,response,script,triplet,typeface
0,31–40 years,2016-05-08 15:16:15 UTC,Letter designer,True,"english,german",True,german,1,0,Daily,latn.i,latin,"['latn.b', 'latn.f', 'latn.i']",pt-sans
1,31–40 years,2016-05-08 15:16:15 UTC,Letter designer,True,"english,german",True,german,1,0,Daily,latn.f,latin,"['latn.b', 'latn.f', 'latn.n']",pt-sans
2,31–40 years,2016-05-08 15:16:15 UTC,Letter designer,True,"english,german",True,german,1,0,Daily,latn.f,latin,"['latn.b', 'latn.f', 'latn.p']",pt-sans
3,31–40 years,2016-05-08 15:16:15 UTC,Letter designer,True,"english,german",True,german,1,0,Daily,latn.f,latin,"['latn.b', 'latn.f', 'latn.q']",pt-sans
4,31–40 years,2016-05-08 15:16:15 UTC,Letter designer,True,"english,german",True,german,1,0,Daily,latn.w,latin,"['latn.b', 'latn.f', 'latn.w']",pt-sans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100067,31–40 years,2017-10-09,Typographer,True,"english,german",True,german,1,1786,"Often, but not daily",latn.x,latin,"['latn.o', 'latn.v', 'latn.x']",century-schoolbook
100068,31–40 years,2017-10-09,Typographer,True,"english,german",True,german,1,1786,"Often, but not daily",latn.r,latin,"['latn.p', 'latn.r', 'latn.v']",century-schoolbook
100069,31–40 years,2017-10-09,Typographer,True,"english,german",True,german,1,1786,"Often, but not daily",latn.p,latin,"['latn.p', 'latn.r', 'latn.x']",century-schoolbook
100070,31–40 years,2017-10-09,Typographer,True,"english,german",True,german,1,1786,"Often, but not daily",latn.p,latin,"['latn.p', 'latn.v', 'latn.x']",century-schoolbook


Imported 100072 rows, 14 columns, 1721 participants, 1787 sessions, 56 trial responses each.


In [118]:
from collections import Counter
from scipy.stats import entropy

def compute_entropy(responses):
    counts = Counter(responses)
    probs = [c / sum(counts.values()) for c in counts.values()]
    return entropy(probs, base=2)

# Compute entropy for each participant
edata = data.groupby("pid")["response"].apply(compute_entropy).reset_index()
edata.columns = ["pid", "entropy"]
# Merge entropy with participants’ data
demographic_data = data.drop_duplicates(subset=["pid"])
edata = pd.merge(edata, demographic_data, on="pid")
# Use data only from the first session
edata = edata[edata["order"] == 1]

In [119]:
# if the distribution is non-normal -> cannot use ANOVA or t-test

pg.normality(edata, group="fluent in script", dv="entropy")

,W,pval,normal
fluent in script,,,
True,0.879936,1.329958e-33,False
False,0.941602,1.654911e-04,False


In [139]:
# Using the Mann Whitney U-statistic instead.
# If the p-value is <0.05 we reject the null hypothesis
# that there is no difference in entropy means

from scipy.stats import mannwhitneyu

for group, value in [
  ("fluent in script", True),
  ("native in script", True),
  ("design skills", "Non-designer"),
]:
  g1 = edata[edata[group] == value]
  g2 = edata[edata[group] != value]
  u, p_val = mannwhitneyu(g1["entropy"], g2["entropy"], alternative="two-sided")
  print(f"{group} ({len(g1)} vs. {len(g2)})")
  print(f"> Mann Whitney U-statistic: {u}, p-value: {p_val:.4f}")


fluent in script (1616 vs. 105)
> Mann Whitney U-statistic: 98095.0, p-value: 0.0072
native in script (1475 vs. 246)
> Mann Whitney U-statistic: 204296.0, p-value: 0.0015
design skills (1101 vs. 620)
> Mann Whitney U-statistic: 367581.0, p-value: 0.0079


In [168]:
import statsmodels.formula.api as smf

# prepare columns for statsmodels/R notation
edata["script"] = edata["script"].astype("category")
edata["fluent_in_script"] = edata["fluent in script"].astype("category")
edata["native_in_script"] = edata["native in script"].astype("category")

model = smf.ols("entropy ~ fluent_in_script + script", data=edata).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                entropy   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     7.031
Date:                Fri, 19 Sep 2025   Prob (F-statistic):           0.000107
Time:                        18:38:14   Log-Likelihood:                 619.48
No. Observations:                1721   AIC:                            -1231.
Df Residuals:                    1717   BIC:                            -1209.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               